In [20]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import json
from random import Random
from torch.utils.data import Dataset
from tqdm import tqdm
import transformers
import traceback
from Annotation_forced import do_annotation
from copy import deepcopy
import itertools
from itertools import product

transformers.logging.set_verbosity_error()

ITEMS_PER_CONDITION = 3

In [22]:
class PromptDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts
    def __len__(self):
        return len(self.prompts)
    def __getitem__(self, idx):
        return self.prompts[idx]
        
models = [
    # model name, batch_size, device, device Mapping
    ("stefan-it/german-gpt2-larger", 2, 0, None),
]

with open("../items/names.json", encoding="utf-8") as nfile:
    namedict = json.load(nfile)
male_names = [name for name in namedict["male"]]
female_names = [name for name in namedict["female"]]
with open("../items/verbs_forced_reference.json", encoding="utf-8") as nfile:
    verbdict = json.load(nfile)
es_verbs = verbdict["es"]
se_verbs = verbdict["se"]

male_pairing = list(product(male_names, female_names, [False]))
female_pairing = list(product(female_names, male_names, [True]))
Random(42).shuffle(male_pairing)
Random(84).shuffle(female_pairing)

conditions = [
    (2,  es_verbs, female_pairing, "NP1"),
    (3,  es_verbs,   male_pairing, "NP1"),
    # (6,  se_verbs, female_pairing, "NP1"),
    # (7,  se_verbs,   male_pairing, "NP1"),
    # (10, es_verbs, female_pairing, "NP2"),
    # (11, es_verbs,   male_pairing, "NP2"),
    (14, se_verbs, female_pairing, "NP2"),
    (15, se_verbs,   male_pairing, "NP2")
]

In [23]:

items_per_condition = []
  
for condition, verbs, pairing, forced_reference in conditions:
    rows = []
    for verbdict in verbs:
        verb, filler, verbclass = verbdict["verb"], verbdict["filler"], verbdict["verbclass"]
        for np1, np2, female in pairing:
            prompt = f"{np1} {verb} {np2}{filler}, weil"
            nrow = {"condition": condition, "type": "Experiment", "prompt": prompt, "NP1": np1, "NP2": np2, 
                    "NP1gender": "f" if female else "m", "verb": verb, "verbclass": verbclass, "forced": forced_reference}
            rows.append(nrow)
    Random(168).shuffle(rows)
    items_per_condition.append(rows)

In [24]:
def make_constraint_function(tokenizer, female):
    weil = tokenizer.encode(", weil")[-2:]
    names = female_names if female else male_names
    pronouns = [" sie", " diese", " jense"] if female else [" er", " dieser", " jener"] 
    tokens = list(map(model.tokenizer.encode, list(map(lambda name: " " + name, names)) + pronouns))
    twos = [items for items in tokens if len(items) > 1]
    twos_one = [item[0] for item in twos]
    twos_two = [item[1] for item in twos]
    all_tokens = list(range(tokenizer.vocab_size))
    tokens = [item[0] for item in tokens]
    def constrainer(batch_id, input_tokens):
        if (input_tokens[-2] == weil[-2]) and (input_tokens[-1] == weil[-1]):
            return tokens
        elif (input_tokens[-3] == weil[-2]) and (input_tokens[-2] == weil[-1]) and (input_tokens[-1] in twos_one):
            return [twos_two[twos_one.index(input_tokens[-1])]]
        else:
            return all_tokens
    return constrainer

In [25]:
for model_name, batch_size, device, device_map in models:
    print(f"now loading: {model_name}")
    model = pipeline("text-generation", model = model_name)
    model.tokenizer.pad_token_id = model.model.config.eos_token_id
    model.tokenizer.padding_side = "left"    
    data = []
    for condition in items_per_condition:
        items = deepcopy(condition)
        constraint_function = make_constraint_function(model.tokenizer, False)
        if ((condition[0]["forced"] == "NP1") and (condition[0]["NP1gender"] == "f")) or ((condition[0]["forced"] == "NP2") and (condition[0]["NP1gender"] == "m")):
            constraint_function = make_constraint_function(model.tokenizer, True)
        bar = tqdm(total = ITEMS_PER_CONDITION)
        bar.set_description(f"Condition {items[0]['condition']}")
        result = []
        counter = 0
        while bar.n < ITEMS_PER_CONDITION:
            counter += 1
            if len(items) >= batch_size:
                rows = pd.DataFrame(items[:batch_size])
                items = items[batch_size:]
                prompts = rows["prompt"].to_list()
                continuations = model(prompts, batch_size=batch_size, remove_invalid_values=True, early_stopping = True, prefix_allowed_tokens_fn=constraint_function, do_sample = False, diversity_penalty = .6, num_beam_groups = 2, num_beams = 2, max_new_tokens = 20)
                continuations = [cont[0]["generated_text"] for cont in continuations]
                continuations = list(map(lambda zipped: zipped[1][len(zipped[0])+1:], zip(prompts, continuations)))
                rows["cont"] = continuations
                res = do_annotation(rows, True)
                res = res[res["Koreferenz"] == res["forced"]]
                result += res.values.tolist()
                bar.update(len(res))
            else:
                print(f"Run out of data in condition {condition[0]['condition']}")
                break
        del bar
        print(f"Generated {counter} sentences for condition {items[0]['condition']}")
        data += result

    exp3 = pd.DataFrame(data, columns = ["condition", "type", "prompt", "NP1", "NP2", "NP1gender", "verb", "verbclass", "forced", "cont", "Koreferenz", "Anaphorische Form"])
    exp3.to_csv(f"../data/forced_coreference--{model_name.replace('/', '--')}.csv", sep=";", index=False)
    
    del model
    del exp3
    del rows
    del items
    
    # FIX ROW BUG

now loading: stefan-it/german-gpt2-larger



Condition 3:   0%|                                                                               | 0/8 [01:15<?, ?it/s]

Condition 2:  67%|███████████████████████████████████████████████▎                       | 2/3 [00:25<00:12, 12.60s/it]
Condition 2: : 4it [00:48, 12.12s/it]                                                                                  


Generated 2 sentences for condition 2


Condition 3: : 4it [00:47, 11.80s/it]                                                                                  


Generated 2 sentences for condition 3


Condition 14: : 4it [00:46, 11.70s/it]                                                                                 


Generated 2 sentences for condition 14


Condition 15: : 4it [00:44, 11.11s/it]                                                                                 

Generated 2 sentences for condition 15


In [20]:
import pandas as pd
from Annotation import do_annotation

In [4]:
df = pd.read_csv("../data/coreference--ai-forever--mGPT.csv", sep=";")

In [9]:
df = df.rename(columns={"np1":"NP1", "np2":"NP2", "cat":"verbclass", "continuation":"cont"})

In [10]:
df["type"] = ["Experiment"] * len(df)

In [14]:
df["cont"] = df.apply(lambda row: row["cont"][len(row["prompt"]) + 1:], axis=1)

In [16]:
df["NP1gender"] = df["female"].apply(lambda b: "f" if b else "m")

In [29]:
"../data/coreference--ai-forever--mGPT.csv"[:-4] + "_annotated.csv"

'../data/coreference--ai-forever--mGPT_annotated.csv'

In [26]:
do_annotation(df[:100], True).to_csv("hi.csv", index=False, sep=";") 